In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

In [ ]:
summarizer1 = pipeline("summarization", model="Falconsai/text_summarization")

In [ ]:
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
from transformers import pipeline

# create pipeline for sentiment analysis
classification = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# model_name = "yangheng/deberta-v3-large-absa-v1.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

# classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
summarizer1 = pipeline("summarization", model="Falconsai/text_summarization")

In [ ]:
topic = pipeline("text2text-generation", model="czearing/article-title-generator")

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import pandas as pd
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import re
import nltk

# Load Spacy English model
nlp = spacy.load("en_core_web_sm")

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()



def clean_text(text):
    """
    Clean the text by removing unnecessary characters and converting to lowercase.
    """
    try:
        # Convert text to lowercase
        text = text.lower()
        # Replace common contractions
        text = re.sub(r"what's", "what is ", text)
        text = text.replace('(ap)', '')
        text = re.sub(r"\'s", " is ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        # Remove non-alphanumeric characters except for '?', '!', and ' '
        text = re.sub(r'[^a-zA-Z?! ]+', '', text)
        # Remove non-ASCII characters
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        # Remove extra spaces and newlines
        text = re.sub('\s+', ' ', text)
        return text
    except Exception as e:
        print("Error cleaning text:", e)
        return ""

def summarizer_fn(article):
    """
    Summarize the article.
    """
    try:
        text = clean_text(article)
        summary = summarizer1(text, max_length=138, min_length=40, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print("Error summarizing article:", e)
        return ""

def mood_rating(article):
    """
    Determine the mood of the article.
    """
    try:
        sentiment = sentiment_model(summarizer_fn(article))
        sentiment_label = sentiment[0]['label']
        sentiment_score = sentiment[0]['score']
        return f"Overall sentiment: {sentiment_label} with score {sentiment_score}"
    except Exception as e:
        print("Error determining mood:", e)
        return ""

def polarity_mood(article):
    """
    Determine the mood of the article using VADER.
    """
    try:
        score = sid.polarity_scores(article)['compound']
        return "Positive" if score >= 0.05 else "Negative" if score <= -0.05 else "Neutral"
    except Exception as e:
        print("Error determining polarity mood:", e)
        return ""

def topic_name(article):
    """
    Generate a topic name for the article.
    """
    try:
        generated_title = topic(article)
        return ("Generated Title:", generated_title[0]['generated_text'])
    except Exception as e:
        print("Error generating topic name:", e)
        return ""

def main(article):
    """
    Perform analysis on the article.
    """
    try:
        # Clean the article text
        cleaned_article = clean_text(article)
        print("Cleaned Article:", cleaned_article)

        # Summarize the article
        summary = summarizer_fn(cleaned_article)
        print("Summary:", summary)

        # Determine the mood of the article
        mood = mood_rating(cleaned_article)
        print("Mood Rating (article):", mood)

        # Determine the mood of the summarized article
        mood1 = mood_rating(summary)
        print("Mood Rating (summary):", mood1)

        # Determine the polarity mood of the article
        mood_polarity = polarity_mood(cleaned_article)
        print("Mood Rating using polarity (article):", mood_polarity)

        # Determine the polarity mood of the summarized article
        mood_polarity1 = polarity_mood(summary)
        print("Mood Rating using polarity (summary):", mood_polarity1)

        # Generate a topic name for the article
        topic = topic_name(cleaned_article)
        print("Topic Name (article):", topic)

        # Generate a topic name for the summarized article
        topic1 = topic_name(summary)
        print("Topic Name (summary):", topic1)

    except Exception as e:
        print("Error in main function:", e)

# User input
user_article = input("Enter the article: ")

# Run the main function
main(user_article)


Enter the article: Whether you dip it, drizzle it, or put it on chicken nuggets or french fries, McDonald’s sauce assortment is expanding this fall.  The fast food chain announced Tuesday that it's adding two new limited-edition sauces to its U.S. menu in two weeks.  The new sauces — McDonald’s Sweet & Spicy Jam and McDonald’s MamboSauce — will hit the menu on Oct. 9, according to a news release from the company.  Are whoppers really too small?Burger King must face whopper of a lawsuit alleging burgers are too small, says judge  What are McDonald's two new sauces?  Here's more about the new limited-edition duo: • Sweet & Spicy Jam: The chain describes it as a breakfast-inspired "jammy red pepper dipping sauce with a tongue-numbing Szechuan peppercorn kick." It also includes apple cider vinegar and cayenne pepper. • MamboSauce: This one is described by McD's as "a tomato-based, sweet, spicy and vinegary sauce, inspired by an Washington, D.C. area sauce staple."  The company came up with

Your max_length is set to 138, but your input_length is only 71. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)


Mood Rating (article): Overall sentiment: negative with score 0.747566282749176
Mood Rating (summary): Overall sentiment: negative with score 0.8576171398162842
Mood Rating using polarity (article): Positive
Mood Rating using polarity (summary): Positive
Topic Name (article): ('Generated Title:', 'mcdonalds adds two new limitededition sauces to its us')
Topic Name (summary): ('Generated Title:', 'Burger King must face whopper of a lawsuit alleging burgers are too small?')
